In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from wordcloud import WordCloud  # Install: pip install wordcloud
import re  # For cleaning text for word cloud

# Optional: If loading from DB
# import psycopg2 # Install: pip install psycopg2-binary


class InsightGenerator:
    """
    Generates insights and visualizations from cleaned and processed mobile banking app reviews.
    """

    def __init__(self, data_source_path="data/sentiment_analysis", load_from_db=False, db_config=None):
        self.data_source_path = data_source_path
        self.load_from_db = load_from_db
        self.db_config = db_config
        self.df = self._load_data()
        self.plots_dir = "reports/visualizations"  # Output directory for plots
        os.makedirs(self.plots_dir, exist_ok=True)
        print(f"Visualizations will be saved to: {self.plots_dir}")
        sns.set_theme(style="whitegrid")  # Apply a nice Seaborn style

    def _load_data(self):
        """
        Loads all processed review data, either from CSVs or from the database.
        Prioritizes database if load_from_db is True and successful.
        """
        df = pd.DataFrame()  # Initialize an empty DataFrame

        if self.load_from_db:
            print("Attempting to load data from PostgreSQL database...")
            if not self.db_config:
                print(
                    "Error: db_config is required for database loading. Falling back to CSV.")
                self.load_from_db = False  # Switch to CSV loading
            else:
                try:
                    # Replace with your actual DB fetching logic (e.g., using psycopg2 directly or ReviewDatabaseManager if you have a fetch_all method)
                    conn = psycopg2.connect(**self.db_config)
                    query = """
                    SELECT r.review_id, r.review_text as review, r.sentiment_label,
                           r.sentiment_score, r.identified_themes, r.rating,
                           r.review_date as date, r.source, b.bank_name as bank
                    FROM reviews r JOIN banks b ON r.bank_id = b.bank_id;
                    """
                    df = pd.read_sql(query, conn)
                    conn.close()
                    print(
                        f"Successfully loaded {len(df)} reviews from the database.")
                    # Ensure 'identified_themes' column is string type for splitting
                    df['identified_themes'] = df['identified_themes'].astype(
                        str)
                except Exception as e:
                    print(
                        f"Error loading data from database: {e}. Falling back to CSV files.")
                    self.load_from_db = False  # Fallback to CSV

        if not self.load_from_db or df.empty:  # If DB loading failed or not chosen
            print(f"Loading data from CSV files in: {self.data_source_path}")
            all_files = glob.glob(os.path.join(
                self.data_source_path, "*_reviews_with_sentiment.csv"))

            if not all_files:
                print(
                    f"No processed CSV files found in '{self.data_source_path}'.")
                print("Please ensure Task 2 was run successfully and generated files.")
                return pd.DataFrame()

            dfs = []
            for filepath in all_files:
                try:
                    df_bank = pd.read_csv(filepath)
                    dfs.append(df_bank)
                    print(
                        f"Loaded {len(df_bank)} reviews from {os.path.basename(filepath)}")
                except Exception as e:
                    print(f"Error loading {filepath}: {e}")

            if not dfs:
                print("No dataframes were successfully loaded from CSVs.")
                return pd.DataFrame()

            df = pd.concat(dfs, ignore_index=True)
            print(f"Total reviews loaded from CSVs: {len(df)}")

        # --- Common Post-Load Processing ---
        df['sentiment_score'] = pd.to_numeric(
            df['sentiment_score'], errors='coerce')
        df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
        df['numerical_sentiment'] = df['sentiment_label'].map(
            {'negative': -1, 'positive': 1, 'neutral': 0})  # Map sentiment labels

        # Drop rows where essential columns for analysis are missing
        return df.dropna(subset=['review', 'rating', 'sentiment_label', 'identified_themes', 'bank'])

    def identify_drivers_and_pain_points(self):
        """
        Identifies key satisfaction drivers (positive themes) and pain points (negative themes).
        Provides illustrative examples from reviews.
        """
        print("\n--- Identifying Drivers and Pain Points ---")
        if self.df.empty:
            print("No data available to identify drivers and pain points.")
            return

        # Explode themes for analysis: each theme becomes a separate row
        # Ensure 'identified_themes' is string, then split by '; '
        df_themes_exploded = self.df.copy()
        df_themes_exploded['theme'] = df_themes_exploded['identified_themes'].astype(
            str).apply(lambda x: [t.strip() for t in x.split('; ') if t.strip()])
        df_themes_exploded = df_themes_exploded.explode('theme')
        # Exclude generic 'Other' category
        df_themes_exploded = df_themes_exploded[df_themes_exploded['theme']
                                                != 'Other/Uncategorized']

        if df_themes_exploded.empty:
            print(
                "No specific themes identified in the data for driver/pain point analysis.")
            return

        # Group by theme and calculate average numerical sentiment
        theme_sentiment = df_themes_exploded.groupby(
            'theme')['numerical_sentiment'].mean().sort_values()

        print(
            "\nAverage Sentiment by Theme (lower = more negative, higher = more positive):")
        print(theme_sentiment)

        # Drivers: Themes with strong positive average sentiment (e.g., > 0.4)
        drivers = theme_sentiment[theme_sentiment >
                                  0.4].tail(2)  # Top 2 most positive
        print("\n**Identified Drivers (Themes with strong positive sentiment):**")
        if drivers.empty:
            print("  (No clear drivers identified based on current threshold. Consider adjusting data or thresholds.)")
        else:
            for theme, score in drivers.items():
                print(f"- **{theme}**: Average Sentiment Score: {score:.2f}")
                # Provide example reviews for evidence
                positive_examples = self.df[
                    (self.df['identified_themes'].str.contains(theme, na=False)) &
                    (self.df['sentiment_label'] == 'positive')
                ]['review'].sample(min(2, len(self.df[(self.df['identified_themes'].str.contains(theme, na=False)) & (self.df['sentiment_label'] == 'positive')]))).tolist()
                if positive_examples:
                    print("  Examples:")
                    for ex in positive_examples:
                        # Print first 100 chars
                        print(f"    - '{ex[:100]}...'")

        # Pain Points: Themes with strong negative average sentiment (e.g., < -0.2)
        # Top 2 most negative
        pain_points = theme_sentiment[theme_sentiment < -0.2].head(2)
        print("\n**Identified Pain Points (Themes with strong negative sentiment):**")
        if pain_points.empty:
            print("  (No clear pain points identified based on current threshold. Consider adjusting data or thresholds.)")
        else:
            for theme, score in pain_points.items():
                print(f"- **{theme}**: Average Sentiment Score: {score:.2f}")
                # Provide example reviews for evidence
                negative_examples = self.df[
                    (self.df['identified_themes'].str.contains(theme, na=False)) &
                    (self.df['sentiment_label'] == 'negative')
                ]['review'].sample(min(2, len(self.df[(self.df['identified_themes'].str.contains(theme, na=False)) & (self.df['sentiment_label'] == 'negative')]))).tolist()
                if negative_examples:
                    print("  Examples:")
                    for ex in negative_examples:
                        # Print first 100 chars
                        print(f"    - '{ex[:100]}...'")

    def compare_banks(self):
        """
        Compares banks based on overall sentiment, average rating, and top themes.
        Addresses "1 driver, 1 pain point per bank" KPI.
        """
        print("\n--- Comparing Banks ---")
        if self.df.empty:
            print("No data available for bank comparison.")
            return

        unique_banks = self.df['bank'].unique()
        if len(unique_banks) < 2:
            print("Not enough unique banks in data for comparison.")
            return

        # Overall Metrics Comparison
        bank_summary = self.df.groupby('bank').agg(
            Average_Rating=('rating', 'mean'),
            Average_Sentiment=('numerical_sentiment', 'mean'),
            Total_Reviews=('review', 'count')
        ).sort_values(by='Average_Rating', ascending=False)
        print("\n**Overall Bank Performance Summary:**")
        print(bank_summary)

        # Drivers and Pain Points per Bank (Minimum Essential KPI)
        print("\n**Drivers and Pain Points Per Bank:**")
        for bank in unique_banks:
            print(f"\n--- {bank} ---")
            bank_df = self.df[self.df['bank'] == bank].copy()
            if bank_df.empty:
                print(f"  No reviews for {bank}.")
                continue

            df_bank_themes_exploded = bank_df.copy()
            df_bank_themes_exploded['theme'] = df_bank_themes_exploded['identified_themes'].astype(
                str).apply(lambda x: [t.strip() for t in x.split('; ') if t.strip()])
            df_bank_themes_exploded = df_bank_themes_exploded.explode('theme')
            df_bank_themes_exploded = df_bank_themes_exploded[
                df_bank_themes_exploded['theme'] != 'Other/Uncategorized']

            if df_bank_themes_exploded.empty:
                print(f"  No specific themes identified for {bank}.")
                continue

            theme_sentiment_bank = df_bank_themes_exploded.groupby(
                'theme')['numerical_sentiment'].mean().sort_values()

            # Driver (most positive theme for this bank)
            bank_driver = theme_sentiment_bank[theme_sentiment_bank > 0].tail(
                1)
            if not bank_driver.empty:
                theme, score = bank_driver.index[0], bank_driver.iloc[0]
                print(f"  Driver: '{theme}' (Avg. Sentiment: {score:.2f})")
                example_review = bank_df[(bank_df['identified_themes'].str.contains(theme, na=False)) & (bank_df['sentiment_label'] == 'positive')]['review'].sample(
                    1).iloc[0] if not bank_df[(bank_df['identified_themes'].str.contains(theme, na=False)) & (bank_df['sentiment_label'] == 'positive')].empty else "No specific example."
                print(f"    Example: '{example_review[:100]}...'")
            else:
                print(f"  No clear driver identified for {bank}.")

            # Pain Point (most negative theme for this bank)
            bank_pain_point = theme_sentiment_bank[theme_sentiment_bank < 0].head(
                1)
            if not bank_pain_point.empty:
                theme, score = bank_pain_point.index[0], bank_pain_point.iloc[0]
                print(f"  Pain Point: '{theme}' (Avg. Sentiment: {score:.2f})")
                example_review = bank_df[(bank_df['identified_themes'].str.contains(theme, na=False)) & (bank_df['sentiment_label'] == 'negative')]['review'].sample(
                    1).iloc[0] if not bank_df[(bank_df['identified_themes'].str.contains(theme, na=False)) & (bank_df['sentiment_label'] == 'negative')].empty else "No specific example."
                print(f"    Example: '{example_review[:100]}...'")
            else:
                print(f"  No clear pain point identified for {bank}.")

    def suggest_improvements(self):
        """
        Suggests practical app improvements based on the identified drivers and pain points.
        Addresses "Practical recommendations" KPI.
        """
        print("\n--- Suggested App Improvements ---")
        if self.df.empty:
            print("No data to suggest improvements.")
            return

        print("Based on common pain points and areas for improvement identified from user reviews:")

        # Example 1: Addressing common app performance issues
        print("\n1. **Improve App Stability and Performance:**")
        print("   - **Evidence:** Frequent mentions of 'crashes', 'freezing', and 'slow loading' often appear in negative reviews (e.g., 'App Stability' theme).")
        print("   - **Recommendation:** Conduct thorough QA testing on various devices and OS versions. Optimize app code for faster load times and smoother navigation. Implement robust error logging to quickly identify and fix bugs.")
        print("   - **KPI Impact:** Reduces user frustration, leading to fewer 1-star ratings and improved overall sentiment, especially for basic usability.")

        # Example 2: Enhancing user experience for core features
        print("\n2. **Streamline Transaction Processes (e.g., Transfers, Bill Payments):**")
        print("   - **Evidence:** Reviews often highlight difficulties with 'fund transfers', 'payment failures', or 'confusing steps' (e.g., 'Transaction & Performance', 'Ease of Use' themes).")
        print("   - **Recommendation:** Simplify the UI/UX for common transactions. Add clearer progress indicators, instant confirmations, and intuitive error messages. Consider features like 'repeat payments' or 'favorite payees' for convenience.")
        print("   - **KPI Impact:** Enhances user satisfaction with critical banking functions, boosting positive sentiment and reducing support queries.")

        # Example 3: Adding value-added features
        print("\n3. **Integrate Personal Financial Management (PFM) Tools:**")
        print("   - **Evidence:** While not directly a pain point, a lack of advanced features for money management is a common gap when comparing modern banking apps. Users might express general desire for more utility beyond basic transactions (could be inferred from positive reviews about 'convenience').")
        print("   - **Recommendation:** Develop features like spending categorization, budgeting tools, savings goal trackers, and investment insights within the app. This adds significant value and encourages users to spend more time within the app.")
        print("   - **KPI Impact:** Increases app stickiness, attracts a broader user base seeking comprehensive financial solutions, and potentially drives higher ratings due to enhanced utility.")

    def visualize_data(self):
        """
        Creates and saves various plots to visualize sentiment, ratings, and themes.
        Saves plots to the 'reports/visualizations' directory.
        """
        print("\n--- Generating Visualizations ---")
        if self.df.empty:
            print("No data available to generate visualizations.")
            return

        # 1. Overall Sentiment Distribution (Bar Chart)
        plt.figure(figsize=(8, 6))
        sns.countplot(data=self.df, x='sentiment_label', palette={
                      'positive': 'skyblue', 'negative': 'lightcoral', 'neutral': 'lightgray'})
        plt.title('Overall Sentiment Distribution of Reviews', fontsize=14)
        plt.xlabel('Sentiment', fontsize=12)
        plt.ylabel('Number of Reviews', fontsize=12)
        plt.tight_layout()
        plt.savefig(os.path.join(self.plots_dir,
                    'overall_sentiment_distribution.png'))
        plt.close()
        print("Saved: overall_sentiment_distribution.png")

        # 2. Rating Distribution (Bar Chart)
        plt.figure(figsize=(8, 6))
        sns.countplot(data=self.df, x='rating', order=sorted(
            self.df['rating'].unique()), palette='viridis')
        plt.title('Distribution of Ratings', fontsize=14)
        plt.xlabel('Rating (Stars)', fontsize=12)
        plt.ylabel('Number of Reviews', fontsize=12)
        plt.tight_layout()
        plt.savefig(os.path.join(self.plots_dir, 'rating_distribution.png'))
        plt.close()
        print("Saved: rating_distribution.png")

        # 3. Average Sentiment Score by Bank (Bar Chart)
        if 'bank' in self.df.columns and self.df['bank'].nunique() > 1:
            plt.figure(figsize=(10, 7))
            sns.barplot(data=self.df, x='bank',
                        y='numerical_sentiment', palette='coolwarm')
            plt.title('Average Sentiment Score by Bank', fontsize=14)
            plt.xlabel('Bank', fontsize=12)
            plt.ylabel('Average Sentiment Score (-1 to 1)', fontsize=12)
            plt.xticks(rotation=45, ha='right', fontsize=10)
            plt.yticks(fontsize=10)
            # Add a line at zero for reference
            plt.axhline(0, color='gray', linewidth=0.8)
            plt.tight_layout()
            plt.savefig(os.path.join(self.plots_dir, 'sentiment_by_bank.png'))
            plt.close()
            print("Saved: sentiment_by_bank.png")
        else:
            print("Skipping 'Sentiment by Bank' plot: Not enough unique banks in data.")

        # 4. Top Themes Overall (Horizontal Bar Chart)
        # Flatten the 'identified_themes' column
        all_themes = self.df['identified_themes'].astype(
            str).str.split('; ').explode().str.strip()
        all_themes = all_themes[all_themes !=
                                'Other/Uncategorized']  # Exclude generic theme
        theme_counts = all_themes.value_counts().head(10)  # Get top 10 themes

        if not theme_counts.empty:
            plt.figure(figsize=(12, 8))
            sns.barplot(x=theme_counts.values,
                        y=theme_counts.index, palette='Blues_d')
            plt.title('Top 10 Most Frequent Themes in Reviews', fontsize=16)
            plt.xlabel('Number of Reviews', fontsize=12)
            plt.ylabel('Theme', fontsize=12)
            plt.xticks(fontsize=10)
            plt.yticks(fontsize=10)
            plt.tight_layout()
            plt.savefig(os.path.join(self.plots_dir, 'top_themes_overall.png'))
            plt.close()
            print("Saved: top_themes_overall.png")
        else:
            print(
                "Skipping 'Top Themes Overall' plot: No specific themes identified or data is empty.")

        # 5. Word Clouds for Positive and Negative Reviews
        # Helper function for text cleaning for word clouds
        def generate_wordcloud_text(df_subset):
            # Combine all reviews, remove non-alphabetic chars, remove short words
            text = " ".join(df_subset['review'].dropna().astype(str).tolist())
            text = re.sub(r'[^a-zA-Z\s]', '', text)
            text = re.sub(r'\b\w{1,2}\b', '', text)
            text = re.sub(r'\s+', ' ', text).strip()
            return text

        positive_reviews_text = generate_wordcloud_text(
            self.df[self.df['sentiment_label'] == 'positive'])
        negative_reviews_text = generate_wordcloud_text(
            self.df[self.df['sentiment_label'] == 'negative'])

        # Generate and save Word Cloud for Positive Reviews
        if positive_reviews_text:
            wordcloud_pos = WordCloud(width=800, height=400, background_color='white',
                                      colormap='Greens').generate(positive_reviews_text)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud_pos, interpolation='bilinear')
            plt.axis('off')
            plt.title('Word Cloud for Positive Reviews', fontsize=14)
            plt.tight_layout()
            plt.savefig(os.path.join(self.plots_dir, 'wordcloud_positive.png'))
            plt.close()
            print("Saved: wordcloud_positive.png")
        else:
            print("Skipping positive word cloud: No positive reviews found.")

        # Generate and save Word Cloud for Negative Reviews
        if negative_reviews_text:
            wordcloud_neg = WordCloud(
                width=800, height=400, background_color='white', colormap='Reds').generate(negative_reviews_text)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud_neg, interpolation='bilinear')
            plt.axis('off')
            plt.title('Word Cloud for Negative Reviews', fontsize=14)
            plt.tight_layout()
            plt.savefig(os.path.join(self.plots_dir, 'wordcloud_negative.png'))
            plt.close()
            print("Saved: wordcloud_negative.png")
        else:
            print("Skipping negative word cloud: No negative reviews found.")

        print("\nAll visualizations generated and saved to the 'reports/visualizations' directory.")

    def note_ethical_considerations(self):
        """
            Notes potential ethical considerations or biases in the review data.
            """
        print("\n--- Ethical Considerations and Potential Biases ---")
        print("When interpreting insights from user reviews, it's crucial to acknowledge potential biases:")
        print("1.  **Selection Bias (Negative Skew):** Users are often more motivated to leave a review when they have a negative experience (a complaint) than a positive one. This can lead to an overrepresentation of negative reviews compared to the true distribution of user experiences.")
        print("2.  **Survivorship Bias:** We only get reviews from users who downloaded, used, and *then* decided to leave a review. Users who had a very bad initial experience and uninstalled immediately might not be represented.")
        print("3.  **Language Nuance and Sarcasm:** Automated sentiment analysis can struggle with complex language, sarcasm, cultural idioms, or reviews containing both positive and negative elements.")
        print("4.  **Lack of Context:** Reviews are often short. A comment like 'Crashes a lot!' lacks context (e.g., on what device, when, how frequently, what specific action caused it).")
        print("5.  **Privacy Concerns:** Ensure that review text, even after anonymization, does not inadvertently reveal personal information if combined with other data points.")
        print("These biases suggest that quantitative insights should be complemented with qualitative review (manual reading) and other data sources (e.g., app analytics, direct user surveys) for a complete picture.")

    def run_insights_pipeline(self):
        print("\n--- Starting Insights and Recommendations Pipeline ---")
        if self.df.empty:
            print(
                "Cannot run insights pipeline: No data loaded. Please check data source.")
            return

        self.identify_drivers_and_pain_points()
        self.compare_banks()
        self.suggest_improvements()
        self.visualize_data()
        self.note_ethical_considerations()

        print("\n--- Insights and Recommendations Pipeline Complete ---")


    # --- Main Execution Block ---
if __name__ == "__main__":
    # --- Configuration ---
    # Path to the folder containing the CSVs generated by Task 2
    PROCESSED_DATA_DIR = "../data/sentiment_analysis"

    # Database configuration (only needed if you want to load data from DB)
    # Make sure psycopg2-binary is installed: pip install psycopg2-binary
    DB_CONFIG = {
        'host': 'localhost',
        'port': '5432',
        'user': 'postgres',
        # <-- !!! IMPORTANT: REPLACE WITH YOUR ACTUAL POSTGRES PASSWORD !!!
        'password': 'postgres',
        'database': 'bank_reviews'
    }

    # Set to True to load from DB, False to load from CSVs (default recommended for simplicity)
    # Change to True if your DB is populated and you prefer to use it
    LOAD_FROM_DATABASE = False

    # --- Instantiate and Run ---
    insight_gen = InsightGenerator(
        data_source_path=PROCESSED_DATA_DIR,
        load_from_db=LOAD_FROM_DATABASE,
        db_config=DB_CONFIG if LOAD_FROM_DATABASE else None
    )
    insight_gen.run_insights_pipeline()

    print("\n--- KPI Checklist ---")
    print("1. 2+ drivers/pain points with evidence: Check the console output under 'Identifying Drivers and Pain Points'.")
    print("2. Clear, labeled visualizations: Check the 'reports/visualizations' directory for generated PNG files.")
    print("3. Practical recommendations: Check the console output under 'Suggested App Improvements'.")
    print("\nRemember to commit visualizations/reports to your 'task-4' branch on GitHub and create your 4-page report.")

Loading data from CSV files in: ../data/sentiment_analysis
Loaded 400 reviews from Bank_of_Abyssinia_Mobile_reviews_with_sentiment.csv
Loaded 400 reviews from Commercial_Bank_of_Ethiopia_Mobile_reviews_with_sentiment.csv
Loaded 400 reviews from Dashen_Bank_Mobile_reviews_with_sentiment.csv
Total reviews loaded from CSVs: 1200
Visualizations will be saved to: reports/visualizations

--- Starting Insights and Recommendations Pipeline ---

--- Identifying Drivers and Pain Points ---

Average Sentiment by Theme (lower = more negative, higher = more positive):
theme
General Negative       -0.931034
App Stability          -0.918919
Developer Options      -0.793103
Network/Connectivity   -0.400000
Login/Authentication   -0.272727
Updates                -0.245283
Transactions           -0.020833
Features                0.042017
Performance             0.159664
Customer Service        0.185185
User Interface          0.381818
Language Support        0.454545
Security                0.476190
Eas

/tmp/ipykernel_433218/2677613527.py:287: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=self.df, x='sentiment_label', palette={


Saved: overall_sentiment_distribution.png


/tmp/ipykernel_433218/2677613527.py:300: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=self.df, x='rating', order=sorted(


Saved: rating_distribution.png


/tmp/ipykernel_433218/2677613527.py:313: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=self.df, x='bank',


Saved: sentiment_by_bank.png


/tmp/ipykernel_433218/2677613527.py:339: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=theme_counts.values,


Saved: top_themes_overall.png
Saved: wordcloud_positive.png
Saved: wordcloud_negative.png

All visualizations generated and saved to the 'reports/visualizations' directory.

--- Ethical Considerations and Potential Biases ---
When interpreting insights from user reviews, it's crucial to acknowledge potential biases:
1.  **Selection Bias (Negative Skew):** Users are often more motivated to leave a review when they have a negative experience (a complaint) than a positive one. This can lead to an overrepresentation of negative reviews compared to the true distribution of user experiences.
2.  **Survivorship Bias:** We only get reviews from users who downloaded, used, and *then* decided to leave a review. Users who had a very bad initial experience and uninstalled immediately might not be represented.
3.  **Language Nuance and Sarcasm:** Automated sentiment analysis can struggle with complex language, sarcasm, cultural idioms, or reviews containing both positive and negative elements.
4.  